In [1]:
from pathlib import Path
import json
from collections import Counter
from tqdm import tqdm

In [2]:
import pandas as pd
import numpy as np 

In [3]:
datadir = Path("../data/2025-02-01-data-citation-corpus-v3.0-json/")

In [4]:
files = list(datadir.glob("*.json"))
data = []
nested_data = {}
def get_row_from_dict(item, corpus_id):
    row = {"corpus_id": corpus_id}
    for k in item.keys():
        if item[k]:
            row[k] = str(type(item[k]))
        else:
            row[k] = "missing"
    return row
    
for fp in tqdm(files):
    txt = fp.read_text()
    records = json.loads(txt)
    for item in records:
        row = get_row_from_dict(item, item["id"])
        data.append(row)
        for k in item.keys():
            if type(item[k]) == dict:
                if k not in nested_data:
                    nested_data[k] = []
                nested_data[k].append(get_row_from_dict(item[k], item["id"]))

df_keys = pd.DataFrame(data)
dfs_nested = {k: pd.DataFrame(v) for k, v in nested_data.items()}

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [01:49<00:00, 18.24s/it]


In [5]:
df_keys

,corpus_id,id,created,updated,repository,publisher,journal,title,dataset,publication,publishedDate,source,affiliations,funders,subjects
0,f08711a2-f72c-485f-b158-cd9a44da26c3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'dict'>,missing,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,missing,missing,<class 'list'>
1,f0871318-164b-426d-bd47-9a90e761f820,<class 'str'>,<class 'str'>,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'dict'>,missing,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,missing,missing,<class 'list'>
2,f0871705-bb17-45dc-b4f2-ff327e17130d,<class 'str'>,<class 'str'>,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'dict'>,missing,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,missing,missing,<class 'list'>
3,f08717de-0174-47cb-89be-027cf445c5b3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'dict'>,<class 'dict'>,missing,<class 'str'>,<class 'str'>,<class 'str'>,missing,<class 'str'>,missing,missing,missing
4,f0872165-1763-48e0-8cb9-5c73c0294ade,<class 'str'>,<class 'str'>,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'dict'>,missing,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,missing,missing,<class 'list'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5322383,909e9304-26bb-4bb2-933c-ebdbbbb5bb0e,<class 'str'>,<class 'str'>,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'dict'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,missing,missing,missing
5322384,909e9cc3-4fc1-40f0-9005-a6b7fb236805,<class 'str'>,<class 'str'>,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'dict'>,missing,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,missing,missing,<class 'list'>
5322385,909e9ff5-d6b2-4c17-8ebf-1bf435c3d43e,<class 'str'>,<class 'str'>,<class 'str'>,<class 'dict'>,missing,missing,missing,<class 'str'>,<class 'str'>,missing,<class 'str'>,missing,missing,<class 'list'>
5322386,909ea2d4-ffcf-4d3f-8dcc-8a06a3256829,<class 'str'>,<class 'str'>,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'dict'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,missing,missing,missing


In [6]:
# metadata completeness table
rows = []
for col in df_keys.columns:
    if col == "corpus_id":
        continue
    num_missing = (df_keys[col] == "missing").sum()
    num_total = len(df_keys[col])
    rows.append({
        "field_name": col,
        "num_missing": num_missing,
        "pct_missing": f"{num_missing/num_total:.2%}",
    })
df_metadata_completeness = pd.DataFrame(rows)

In [7]:
print(f"{len(df_keys):,} records")

5,322,388 records


In [8]:
dfs_nested.keys()

dict_keys(['repository', 'publisher', 'journal'])

In [9]:
df_metadata_completeness

,field_name,num_missing,pct_missing
0,id,0,0.00%
1,created,0,0.00%
2,updated,0,0.00%
3,repository,355970,6.69%
4,publisher,116472,2.19%
5,journal,175881,3.30%
6,title,4062295,76.32%
7,dataset,0,0.00%
8,publication,0,0.00%
9,publishedDate,176678,3.32%
